In [22]:
import pandas as pd
import nltk
nltk.download('rslp')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package rslp to /home/ynx/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ynx/nltk_data...


In [23]:
df = pd.read_csv('./dados_treino_final_sem_neutro_ajustado.csv', sep=';')
df.shape

(359, 7)

In [24]:
token_espaco=nltk.tokenize.WhitespaceTokenizer()
frase_processada=list()
for noticia in df.texto_noticia:
    nova_frase=list()
    palavras_texto=token_espaco.tokenize(noticia)

In [25]:
stemmer = nltk.RSLPStemmer()
for palavra in palavras_texto:
    nova_frase.append(stemmer.stem(palavra))
    frase_processada.append(" ".join(nova_frase))
wordnet_lemmatizer = WordNetLemmatizer()
for palavra in palavras_texto:
    nova_frase.append(wordnet_lemmatizer.lemmatize(palavra))
    frase_processada.append(" ".join(nova_frase))

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
vetorizar=CountVectorizer(max_features=50)
bag_of_words=vetorizar.fit_transform(texto[coluna_texto])

NameError: name 'texto' is not defined

In [ ]:
def pareto(texto, coluna_texto, quantidade):
    todas_palavras=' '.join([texto for texto in   texto[coluna_texto]])
    frequencia = nltk.FreqDist(token_espaco.tokenize(todas_palavras))
    df_frequencia=pd.DataFrame({'Palavra':list(frequencia.keys()),
    "Frequencia":list(frequencia.values())})
    plt.figure(figsize=(12,8))
    ax=sns.barplot(data=df_frequencia.nlargest(columns="Frequencia",  n=10), x="Palavra", y="Frequencia", color="gray")
    ax.set(ylabel="Contagem")
    plt.show()